In [1]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   
from pyntcloud import PyntCloud

#Loading the point cloud

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedCloud.ply"))




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Making a voxelgrid from the point cloud
voxel_size = 3
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [3]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [ ]:
#This is old code, not if needed anymore
from sklearn.decomposition import PCA
from tqdm.auto import tqdm

#Extracting points from each voxel, calculating features for all of them and storing them in lists
voxel_size = 3


linearity =[]
planarity = []
scattering = []
omnivariance = []
sum_eigenvalues = []
anisotropy = []
eigentropy = []
change_of_curvature = []
points_per_voxel = []
z_range = []
 

#Calculating Z-range and standard deviation from the plane for each voxel

#Z-range

#Defining a function to calculate the z range
def calculateZRange(points):
    if len(points) < 15:
        return 0
    return(np.max(points[:,2]) - np.min(points[:,2]))
   
#Run this to just run it on the first 100 voxels
#voxel_center_coordinates = voxel_center_coordinates[:100]


for center in tqdm(voxel_center_coordinates):
    minboundx = center[0] - voxel_size/2
    minboundy = center[1] - voxel_size/2
    minboundz = center[2] - voxel_size/2

    maxboundx = center[0] + voxel_size/2
    maxboundy = center[1] + voxel_size/2
    maxboundz = center[2] + voxel_size/2

    voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
    #print("Number of points in voxel ", len(voxel_extracted_points.points))
    
    points = np.asarray(voxel_extracted_points.points)
    
    #Checking if there are enough points in the voxel to calculate PCA
    
    if len(points) < 15:
        linearity.append(0)
        planarity.append(0)
        scattering.append(0)
        omnivariance.append(0)
        sum_eigenvalues.append(0)
        anisotropy.append(0)
        eigentropy.append(0)
        change_of_curvature.append(0)
        points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
        continue
    
    pca = PCA(n_components=3)
    pca.fit(points)

    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    
   
    

In [7]:
#Defining a function to extract points from a voxel based on center coordinates. Returns a list of points

from sklearn.decomposition import PCA
from tqdm.auto import tqdm


def extractPointCloudFromVoxel(voxel_center_coordinates, pointCloud, voxel_size):
    extractedPointClouds = []
    for center in tqdm(voxel_center_coordinates):
        minboundx = center[0] - voxel_size/2
        minboundy = center[1] - voxel_size/2
        minboundz = center[2] - voxel_size/2

        maxboundx = center[0] + voxel_size/2
        maxboundy = center[1] + voxel_size/2
        maxboundz = center[2] + voxel_size/2

        pcd = pointCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
        #print("Number of points in voxel ", len(voxel_extracted_points.points))
        extractedPointClouds.append(pcd)
    return extractedPointClouds
    
    
    

In [21]:
#Function to calculate features from a list of points clouds (could be one or more), returns lists for each feature
def calculatePCAFeatures(listOfPointClouds):
    
    linearity =[]
    planarity = []
    scattering = []
    omnivariance = []
    sum_eigenvalues = []
    anisotropy = []
    eigentropy = []
    change_of_curvature = []
    eigenvaluesList = []
    eigenvectorsList = []
    points_per_voxel = []
    
    for pcd in tqdm(listOfPointClouds):
        points = pcd.points
    
        if len(points) < 15:
            linearity.append(0)
            planarity.append(0)
            scattering.append(0)
            omnivariance.append(0)
            sum_eigenvalues.append(0)
            anisotropy.append(0)
            eigentropy.append(0)
            change_of_curvature.append(0)
            points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
            continue
    
    
        pca = PCA(n_components=3)
        pca.fit(points)

    # Extract the eigenvalues and eigenvectors
        eigenvalues = pca.explained_variance_
        eigenvectors = pca.components_
    
      #Calculating the features
        l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
        p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
        s = eigenvalues[2] / eigenvalues[0]
        o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
        s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
        a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
        e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
        c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])

        linearity.append(l)
        planarity.append(p)
        scattering.append(s)
        omnivariance.append(o)
        sum_eigenvalues.append(s_e)
        anisotropy.append(a)
        eigentropy.append(e)
        change_of_curvature.append(c_o_c)
        eigenvaluesList.append(eigenvalues)
        eigenvectorsList.append(eigenvectors)
        points_per_voxel.append(points)
        
    return linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvaluesList, eigenvectorsList, points_per_voxel

In [24]:
#Defining a seperate function to calculate z-range for a set of points
def calculateZRange(points):
    z_range = []
    if len(points) < 15:
        z_range.append(0)
        z = np.max(points[:,2]) - np.min(points[:,2])
        z_range.append(z)
    return z_range
   

In [8]:
#Extracting the points from each voxel

extractedPointClouds = extractPointCloudFromVoxel(voxel_center_coordinates, initialCloud, voxel_size)

  0%|          | 0/29528 [00:00<?, ?it/s]

In [22]:
#Calculating the features for each voxel
linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvalues, eigenvectors, points_per_voxel = calculatePCAFeatures(extractedPointClouds)

  0%|          | 0/29528 [00:00<?, ?it/s]

In [25]:
#Calculating z-range for each voxel

z_range = []
for pcd in tqdm(extractedPointClouds):
    points = np.asarray(pcd.points)
    z = calculateZRange(points)
    z_range.append(z)


  0%|          | 0/29528 [00:00<?, ?it/s]

In [118]:
#Extracting points based on two different parameters




            

In [ ]:
#Creating a function to calculate the features for a given set of points

def calculateFeatures(points):
    pts = np.asarray(points) 
    pca = PCA(n_components=3)
    pca.fit(pts)
    
    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0]
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    return l, p, s, o, s_e, a, e, c_o_c, eigenvalues, eigenvectors



In [ ]:
#Calculating features for 3 houses
house =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse.ply"))
house2 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse2.ply"))
house3 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse3.ply"))

housePoints = np.asarray(house.points)
housePoints2 = np.asarray(house2.points)
housePoints3 = np.asarray(house3.points)


In [ ]:
#Calculating features for trees
tree =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree.ply"))
tree2 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree2.ply"))
tree3 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree3.ply"))
tree4 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree4.ply"))
tree5 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree5.ply"))

trees = [tree, tree2, tree3, tree4, tree5]
treeFeatures=[]



In [ ]:
print(calculateFeatures(trees[0].points))

In [ ]:
def calculateAverageFeatures(feature):
    average = 0
    for i in feature:
        average += i
    return average/len(feature)